In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random
import re
import os
from urllib.parse import urljoin

# Define headers to mimic a browser request
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept': 'text/html,application/xhtml+xml,application/xml',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Lists to store data for each CSV file
genre_data = []
prime_data = []
region_data = []

def get_soup(url):
    """Get BeautifulSoup object from URL with error handling and retries"""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"Requesting {url}")
            response = requests.get(url, headers=HEADERS)
            if response.status_code == 200:
                print(f"Response received: {len(response.text)} characters")
                return BeautifulSoup(response.text, 'html.parser')
            else:
                print(f"Non-200 status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")

        if attempt < max_retries - 1:
            wait_time = random.uniform(3, 7)
            print(f"Retrying in {wait_time:.2f} seconds...")
            time.sleep(wait_time)
        else:
            print("Max retries reached. Skipping this URL.")
            return None

def extract_year(text):
    """Extract year from text using regex"""
    if not text:
        return -1

    year_match = re.search(r'(\d{4})', text)
    if year_match:
        return int(year_match.group(1))
    return -1

def extract_runtime(text):
    """Extract runtime in minutes from text"""
    if not text:
        return -1

    runtime_match = re.search(r'(\d+)\s*(?:hr|hour|h)?(?:\s+(\d+)\s*(?:min|m))?|(\d+)\s*(?:min|m)', text)
    if runtime_match:
        hours = int(runtime_match.group(1) or 0)
        minutes = int(runtime_match.group(2) or runtime_match.group(3) or 0)
        return hours * 60 + minutes
    return -1

def extract_votes(text):
    """Extract number of votes from text"""
    if not text:
        return -1

    clean_text = re.sub(r'[^\d]', '', text)
    if clean_text:
        return int(clean_text)
    return -1

def scrape_movie_details(movie_url, data_id):
    """Scrape detailed information about a movie or TV show"""
    print(f"Scraping: {movie_url}")
    soup = get_soup(movie_url)
    if not soup:
        return False

    try:
        # Extract title
        title_elem = soup.select_one('h1[data-testid="hero__pageTitle"], h1.sc-b73cd867-0, h1.TitleHeader__TitleText')
        title = title_elem.text.strip() if title_elem else "Unknown"

        # Determine content type (movie or TV show)
        content_type = "movie"
        if soup.find('a', href=lambda x: x and ('episodes' in x or 'seasons' in x)):
            content_type = "tvSeries"
        elif soup.find('span', string=lambda x: x and ('Episode' in x or 'TV Series' in x)):
            content_type = "tvSeries"

        # Extract release year and end year (for TV series)
        release_year = -1
        end_year = -1

        # Try different selectors for year information
        year_container = soup.select_one('ul[data-testid="hero-title-block__metadata"], .TitleBlockMetaData')
        if year_container:
            year_text = year_container.text.strip()
            years = re.search(r'(\d{4})(?:\s*[–-]\s*(\d{4}|\s*))?', year_text)
            if years:
                release_year = int(years.group(1)) if years.group(1) else -1
                end_year = int(years.group(2)) if years.group(2) and years.group(2).strip() else -1

        # If we couldn't extract the year, try a more generic approach
        if release_year == -1:
            # Look for any element with a 4-digit number that could be a year
            for element in soup.select('span, div, li'):
                if element.text and len(element.text) < 30:  # Short text elements
                    year_match = re.search(r'(\d{4})', element.text)
                    if year_match:
                        release_year = int(year_match.group(1))
                        break

        # Extract runtime
        length = -1
        runtime_container = soup.select_one('[data-testid="hero-title-block__metadata"], .TitleBlockMetaData')
        if runtime_container:
            runtime_text = runtime_container.text
            length = extract_runtime(runtime_text)

        # Alternative method for runtime
        if length == -1:
            runtime_elem = soup.select_one('[data-testid="title-techspec-runtime"], .TechSpec__MetaItem')
            if runtime_elem:
                runtime_text = runtime_elem.text
                length = extract_runtime(runtime_text)

        # Extract rating
        rating = -1
        rating_elem = soup.select_one('[data-testid="hero-rating-bar__aggregate-rating__score"] span, .RatingBar__RatingContainer span, .AggregateRatingButton__RatingScore')
        if rating_elem:
            try:
                rating = float(rating_elem.text)
            except ValueError:
                pass

        # Extract votes
        votes = -1
        votes_elem = soup.select_one('[data-testid="hero-rating-bar__aggregate-rating__score"] + div, .AggregateRatingButton__TotalRatingAmount')
        if votes_elem:
            votes = extract_votes(votes_elem.text)

        # Extract certificate/content rating
        certificate = None
        cert_elem = soup.select_one('a[href*="certificates"], .TitleBlockMetaData span')
        if cert_elem:
            certificate = cert_elem.text.strip()

        # Extract description/plot
        description = "Unknown"
        plot_elem = soup.select_one('[data-testid="plot-xl"], [data-testid="plot-l"], .GenresAndPlot__TextContainerBreakpointXL, .Plot__PlotText')
        if plot_elem:
            description = plot_elem.text.strip()

        # Extract gross revenue (if available)
        gross = -1
        box_office_section = soup.find('h3', string=lambda s: s and 'Box office' in s)
        if box_office_section:
            gross_elem = box_office_section.find_next('div')
            if gross_elem:
                gross_text = gross_elem.text.strip()
                if gross_text:
                    amount = re.search(r'[\$£€]?\s*(\d+(?:[.,]\d+)?)\s*(?:million|billion|M|B)?', gross_text)
                    if amount:
                        value = float(amount.group(1).replace(',', ''))
                        if 'billion' in gross_text or 'B' in gross_text:
                            value *= 1000
                        gross = value

        # Extract genres - FIXED SECTION
        genres = []

        # Method 1: Try to extract genres from structured data (JSON-LD)
        script_tags = soup.find_all('script', {'type': 'application/ld+json'})
        for script in script_tags:
            try:
                import json
                data = json.loads(script.string)
                if 'genre' in data:
                    if isinstance(data['genre'], list):
                        for genre in data['genre']:
                            genres.append(genre)
                    elif isinstance(data['genre'], str):
                        genres.append(data['genre'])
            except (json.JSONDecodeError, AttributeError, TypeError):
                pass

        # Method 2: Try various selectors for genre elements
        if not genres:
            # Look for elements with "genres" in the URL
            genre_links = soup.select('a[href*="genres"]')
            for link in genre_links:
                genre = link.text.strip()
                if genre and len(genre) < 20 and genre not in genres:
                    genres.append(genre)

        # Method 3: Search for spans/divs that might contain genre information
        if not genres:
            common_genres = [
                'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
                'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                'Sport', 'Thriller', 'War', 'Western'
            ]
            for span in soup.find_all(['span', 'div']):
                text = span.text.strip()
                if text in common_genres and text not in genres:
                    genres.append(text)

        # Method 4: Look for elements with specific class names that often contain genres
        if not genres:
            genre_containers = soup.select('.genres, .genre, [class*="Genre"], [class*="genre"]')
            for container in genre_containers:
                for genre in container.text.split(','):
                    clean_genre = genre.strip()
                    if clean_genre and len(clean_genre) < 20 and clean_genre not in genres:
                        genres.append(clean_genre)

        # Add genres to genre_data
        print(f"Found genres: {genres}")
        for genre in genres:
            genre_data.append({
                'dataId': data_id,
                'genre': genre
            })

        # Extract region/country
        regions = []
        country_elements = soup.select('a[href*="country_of_origin"], .ipc-metadata-list-item__list-content-item[href*="country"], .TextLink[href*="country"]')

        if not country_elements:
            details_section = soup.find('section', {'data-testid': 'Details'})
            if details_section:
                country_elements = details_section.select('a[href*="country"]')

        # If still no country elements, try to extract from JSON-LD
        if not country_elements:
            for script in script_tags:
                try:
                    import json
                    data = json.loads(script.string)
                    if 'countryOfOrigin' in data and 'name' in data['countryOfOrigin']:
                        regions.append(data['countryOfOrigin']['name'])
                except (json.JSONDecodeError, AttributeError, TypeError):
                    pass

        for country_elem in country_elements:
            region = country_elem.text.strip()
            if region and region not in regions:
                regions.append(region)

        # If still no regions found, add a default "Unknown" region
        if not regions:
            regions.append("India")  # Since we're scraping Indian movies, assume India by default

        # Add regions to region_data
        for region in regions:
            region_data.append({
                'dataId': data_id,
                'region': region
            })

        # Add to prime_data
        prime_data.append({
            'dataId': data_id,
            'contentType': content_type,
            'title': title,
            'length': length,
            'releaseYear': release_year,
            'endYear': end_year,
            'votes': votes,
            'rating': rating,
            'gross': gross,
            'certificate': certificate,
            'description': description
        })

        print(f"Successfully scraped: {title} ({release_year})")
        return True

    except Exception as e:
        print(f"Error processing {movie_url}: {e}")
        return False

def scrape_imdb_list(url, max_items=100):
    """Scrape a list of movies/shows from an IMDb list page"""
    print(f"Fetching list page: {url}")
    soup = get_soup(url)
    if not soup:
        print("Failed to get soup for list page")
        return []

    movie_links = []
    base_url = "https://www.imdb.com"

    # Special case for Indian movies list
    if "india/top-rated-indian-movies" in url:
        print("Detected Indian movies list, using specific selectors")
        title_elements = soup.select('.lister-item-header a')

        if not title_elements:
            print("Trying alternative Indian movies selector")
            title_elements = soup.select('.titleColumn a')
    else:
        # Try multiple selectors for different IMDb page formats
        title_elements = soup.select('div.ipc-metadata-list-summary-item__tc a[href^="/title"]')

        if not title_elements:
            title_elements = soup.select('td.titleColumn a')

        if not title_elements:
            title_elements = soup.select('a.ipc-title-link-wrapper[href^="/title"]')

    # If we still haven't found any, try the most generic selector
    if not title_elements:
        print("Trying generic selector for movie links")
        title_elements = soup.select('a[href^="/title/tt"]')

    print(f"Found {len(title_elements)} title elements")

    for elem in title_elements[:max_items]:
        if 'href' in elem.attrs:
            href = elem['href']
            movie_id_match = re.search(r'/title/(tt\d+)', href)
            if movie_id_match:
                movie_id = movie_id_match.group(1)
                movie_url = f"{base_url}/title/{movie_id}/"
                print(f"Found movie link: {movie_url}")
                movie_links.append(movie_url)

    return movie_links

def main():
    """Main function to orchestrate the scraping process"""
    # URLs to scrape from
    urls = [
        'https://www.imdb.com/india/top-rated-indian-movies/?ref_=fea_eds_center-1_india_tr_india250_cta',
        # Add more URLs as needed
    ]

    data_id = 100000
    all_scraped_links = set()

    for url in urls:
        print(f"\n=== Scraping list: {url} ===\n")
        movie_links = scrape_imdb_list(url, max_items=250)

        print(f"Found {len(movie_links)} movies to scrape")

        if not movie_links:
            print("No movie links found. Skipping to next URL.")
            continue

        for link in movie_links:
            if link in all_scraped_links:
                print(f"Skipping duplicate: {link}")
                continue

            all_scraped_links.add(link)
            success = scrape_movie_details(link, data_id)

            if success:
                data_id += 1

            # Random delay between requests
            time.sleep(random.uniform(3, 7))

    # Create DataFrames
    genre_df = pd.DataFrame(genre_data)
    prime_df = pd.DataFrame(prime_data)
    region_df = pd.DataFrame(region_data)

    # Create output directory if it doesn't exist
    os.makedirs('imdb_data', exist_ok=True)

    # Save to CSV files
    genre_df.to_csv('imdb_data/contentDataGenre.csv', index=False)
    prime_df.to_csv('imdb_data/contentDataPrime.csv', index=False)
    region_df.to_csv('imdb_data/contentDataRegion.csv', index=False)

    print("\n=== Scraping Complete ===")
    print(f"Scraped {len(prime_data)} movies/shows")
    print(f"Generated {len(genre_data)} genre entries")
    print(f"Generated {len(region_data)} region entries")
    print("Files saved in 'imdb_data' folder: contentDataGenre.csv, contentDataPrime.csv, contentDataRegion.csv")

if __name__ == "__main__":
    main()


=== Scraping list: https://www.imdb.com/india/top-rated-indian-movies/?ref_=fea_eds_center-1_india_tr_india250_cta ===

Fetching list page: https://www.imdb.com/india/top-rated-indian-movies/?ref_=fea_eds_center-1_india_tr_india250_cta
Requesting https://www.imdb.com/india/top-rated-indian-movies/?ref_=fea_eds_center-1_india_tr_india250_cta
Response received: 2326622 characters
Detected Indian movies list, using specific selectors
Trying alternative Indian movies selector
Trying generic selector for movie links
Found 250 title elements
Found movie link: https://www.imdb.com/title/tt23849204/
Found movie link: https://www.imdb.com/title/tt0079221/
Found movie link: https://www.imdb.com/title/tt0093603/
Found movie link: https://www.imdb.com/title/tt0367495/
Found movie link: https://www.imdb.com/title/tt0052572/
Found movie link: https://www.imdb.com/title/tt8176054/
Found movie link: https://www.imdb.com/title/tt1187043/
Found movie link: https://www.imdb.com/title/tt10534500/
Found m